In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

# import os
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"


tfds.disable_progress_bar()

In [2]:
tf.__version__

'2.10.0'

In [3]:
devices = tf.config.experimental.list_physical_devices('GPU')
devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(devices[0], True)
    print("Success")
except:
    print("Exception occured")
    pass

Success


**Read more about this dataset here: https://ai.stanford.edu/~amaas/data/sentiment/
As per this article:
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.** 

In [6]:
dataset, info = tfds.load('imdb_reviews', data_dir='./datasets', with_info=True, as_supervised=True)

In [7]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='./datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        'train': <Sp

In [8]:
dataset

{'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [9]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [10]:
type(train_dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [11]:
len(train_dataset)

25000

In [12]:
len(test_dataset)

25000

In [13]:
for sample in train_dataset:
    print(sample[0].numpy())
    print(sample[1].numpy())
    break

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


2022-10-30 13:18:59.871625: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-30 13:18:59.886872: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [15]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [16]:
for example, label in train_dataset.take(1):
    print('texts: ', example.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

texts:  [b'THE WATERDANCE (1991) The main character of The Waterdance, played by Eric Stoltz, finds himself in a rehab center with some others similarly injured. And there he must face an harsh new life, confined to a wheelchair. It\'s an interesting, and promising premise, but unfortunately, it fails to deploy. What ensues instead is largely Hollywood schmaltz, with some interesting moments. Certainly the cast (Eric Stoltz, William Forsythe, Wesley Snipes, et al) is brilliant, and perform well here as one would expect, but their talents are wasted. The characters are mainly stereotypes of one kind or another, and most of them are thoroughly unlikeable (the Snipes character being the exception). I suppose this is some kind of attempt to break through people\'s ideas about the handicapped being "crippled" or "weak", by depicting them, for the most part, as in-your-face pricks, but it makes for an entirely annoying experience. Admittedly it will show you something of what those with perm

2022-10-30 13:19:00.190780: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [17]:
e = tf.keras.layers.experimental.preprocessing.TextVectorization()
e.adapt([
    "I love samosas and jalebi",
    "I love biking and yoga",
    "I love tensorflow"
])

2022-10-30 13:19:00.296083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [18]:
e.get_vocabulary()

['',
 '[UNK]',
 'love',
 'i',
 'and',
 'yoga',
 'tensorflow',
 'samosas',
 'jalebi',
 'biking']

In [19]:
e(["I love pizza"]).numpy()

array([[3, 2, 1]])

In [20]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

2022-10-30 13:19:00.473526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [21]:
vocab = np.array(encoder.get_vocabulary())
vocab[:25]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but',
       'film', 'on', 'not', 'you', 'are'], dtype='<U14')

In [22]:
example[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'THE WATERDANCE (1991) The main character of The Waterdance, played by Eric Stoltz, finds himself in a rehab center with some others similarly injured. And there he must face an harsh new life, confined to a wheelchair. It\'s an interesting, and promising premise, but unfortunately, it fails to deploy. What ensues instead is largely Hollywood schmaltz, with some interesting moments. Certainly the cast (Eric Stoltz, William Forsythe, Wesley Snipes, et al) is brilliant, and perform well here as one would expect, but their talents are wasted. The characters are mainly stereotypes of one kind or another, and most of them are thoroughly unlikeable (the Snipes character being the exception). I suppose this is some kind of attempt to break through people\'s ideas about the handicapped being "crippled" or "weak", by depicting them, for the most part, as in-your-face pricks, but it makes for an entirely annoying experience. Admittedly it will

In [23]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[  2,   1,   1, ...,   0,   0,   0],
       [  1,   1,   2, ...,   0,   0,   0],
       [ 10, 284,  43, ...,   0,   0,   0]])

In [24]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

Original:  b'THE WATERDANCE (1991) The main character of The Waterdance, played by Eric Stoltz, finds himself in a rehab center with some others similarly injured. And there he must face an harsh new life, confined to a wheelchair. It\'s an interesting, and promising premise, but unfortunately, it fails to deploy. What ensues instead is largely Hollywood schmaltz, with some interesting moments. Certainly the cast (Eric Stoltz, William Forsythe, Wesley Snipes, et al) is brilliant, and perform well here as one would expect, but their talents are wasted. The characters are mainly stereotypes of one kind or another, and most of them are thoroughly unlikeable (the Snipes character being the exception). I suppose this is some kind of attempt to break through people\'s ideas about the handicapped being "crippled" or "weak", by depicting them, for the most part, as in-your-face pricks, but it makes for an entirely annoying experience. Admittedly it will show you something of what those with pe

In [25]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [26]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
sample_text = ('awesome movie, I loved it so much')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

2022-10-30 13:19:04.321459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
[0.00827623]


2022-10-30 13:19:04.549864: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-30 13:19:04.612755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [27]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [28]:

model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10


2022-10-30 13:19:07.955646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-30 13:19:08.459314: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2022-10-30 13:19:08.464257: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-30 13:19:08.615670: I tensorflow/co

 35/391 [=>............................] - ETA: 1:38:05 - loss: 0.6939 - accuracy: 0.5022

In [ ]:
import sys
print(sys.version)

3.8.5 (tags/v3.8.5:580fbb0, Jul 20 2020, 15:57:54) [MSC v.1924 64 bit (AMD64)]
